In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from time import perf_counter

In [2]:
filename = './datasets/partitionTest.txt'

In [4]:
t0 = perf_counter()
testgraph = nx.read_edgelist(filename, comments='#', delimiter=None, create_using=None, nodetype=None, data=True, edgetype=None, encoding='utf-8')
t_load = perf_counter() - t0

In [5]:
print(int(sum(nx.triangles(testgraph).values())/3))

3


In [6]:
full_adj = nx.adjacency_matrix(testgraph)
print(full_adj.shape)

(6, 6)


/tmp/ipykernel_772469/1967707738.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  full_adj = nx.adjacency_matrix(testgraph)


In [7]:
print(full_adj)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1, 0)	1
  (1, 4)	1
  (2, 0)	1
  (2, 3)	1
  (2, 5)	1
  (3, 0)	1
  (3, 2)	1
  (3, 4)	1
  (3, 5)	1
  (4, 1)	1
  (4, 3)	1
  (4, 5)	1
  (5, 2)	1
  (5, 3)	1
  (5, 4)	1


In [8]:
full_adj_dense = full_adj.todense()
print(full_adj_dense.shape)

(6, 6)


In [9]:
print(full_adj_dense)

[[0 1 1 1 0 0]
 [1 0 0 0 1 0]
 [1 0 0 1 0 1]
 [1 0 1 0 1 1]
 [0 1 0 1 0 1]
 [0 0 1 1 1 0]]


In [10]:
for i in range(0, full_adj_dense.shape[0]):
    for j in range(0, full_adj_dense.shape[1]):
        if i >= j:
            full_adj_dense[i, j] = 0

In [11]:
print(full_adj_dense)

[[0 1 1 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]


In [13]:
part_1_adj = full_adj_dense[:, 0:3]
part_2_adj = full_adj_dense[:, 3:full_adj_dense.shape[1]]
print(part_1_adj.shape[0], part_1_adj.shape[1])
print(part_2_adj.shape)

6 3
(6, 3)


In [14]:
part_1_matrix = np.zeros((full_adj.shape))
part_2_matrix = np.zeros((full_adj.shape))
print(part_1_matrix.shape)
print(part_2_matrix.shape)

(6, 6)
(6, 6)


In [15]:
for i in range (0, part_1_adj.shape[0]):
    for j in range(0, part_1_adj.shape[1]):
        part_1_matrix[i,j] = part_1_adj[i,j]
        
for i in range (0, part_2_adj.shape[0]):
    for j in range(0, part_2_adj.shape[1]):
        part_2_matrix[i, j+3] = part_2_adj[i,j]

In [16]:
print(part_1_matrix)
print(part_2_matrix)

[[0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]]


In [17]:
part_1_graph = nx.from_numpy_matrix(part_1_matrix)
part_2_graph = nx.from_numpy_matrix(part_2_matrix)

In [20]:
count = 0
triangle = []
for edge_list in testgraph.edges:
#     if int(edge_list[1]) < 2000:
    srcSet = part_1_graph.adj[int(edge_list[0])]
    dstSet = part_1_graph.adj[int(edge_list[1])]
    for vertexID in dstSet:
        if (vertexID in srcSet) and (vertexID > int(edge_list[1])):
            count += 1
            triangle.append((edge_list[0], edge_list[1], vertexID))
print("part_1 triangles: ", count) 

part_1 triangles:  0


In [19]:
count_1 = 0
triangle_1 = []
for edge_list in testgraph.edges:
#     if int(edge_list[1]) >= 2000:
    srcSet = part_2_graph.adj[int(edge_list[0])]
    dstSet = part_2_graph.adj[int(edge_list[1])]
    for vertexID in dstSet:
        if (vertexID in srcSet) and (vertexID > int(edge_list[1])):
            count_1 += 1
            triangle_1.append((edge_list[0], edge_list[1], vertexID))
print("part_2 triangles: ", count_1)
print("Triangles in total:", count + count_1)

part_2 triangles:  7
Triangles in total: 7
